# Assignment Set 10
by **Amir Mehrpanah** for Deep Learing Course at Data Science Center, SBU

In [ ]:
#@title Data Injection
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_ds = torchvision.datasets.MNIST(root ='data/',
                                      train=True,
                                      download=True,
                                      transform=data_transform)

val_ds = torchvision.datasets.MNIST(root ='data/',
                                     train=False,
                                    download=True,
                                    transform=data_transform)

In [ ]:
#@title Define The Model
loss_function = nn.CrossEntropyLoss()
class CNN3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.pool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(800, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = nn.Dropout(p=0.3)(x)        
        x = torch.softmax(self.fc2(x),dim=1)
        return x

In [ ]:
def train(trial,epochs,optimizer,training_generator,cnn,device):
    epoch_loss = []
    epoch_acc = []
    for epoch in range(epochs):
        running_loss = []
        running_acc = []
        for local_batch, local_labels in training_generator:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            optimizer.zero_grad()
            outputs = cnn(local_batch)
            loss = loss_function(outputs, local_labels)
            loss.backward()
            optimizer.step()
            running_loss += [loss.item()]
            _, predicted = torch.max(outputs, 1)
            running_acc += [(predicted == local_labels).sum().item()/training_generator.batch_size]
        
        epoch_acc.append(np.mean(running_acc))
        epoch_loss.append(np.mean(running_loss))
        print('[{:d}] loss: {:7.5f} - acc {:7.5f}'.format(epoch + 1,epoch_loss[-1],epoch_acc[-1]))
        if len(epoch_loss) > 4:
            print('var: ',np.var(epoch_loss[-5:]))
            trial.report(np.var(epoch_loss[-5:]), epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()
                
    return epoch_loss[-1]

In [ ]:
opt_functional = [optim.Adam,
           optim.RMSprop,
           optim.SGD]
def objective(trial):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    print(device)
    torch.backends.cudnn.benchmark = True
    opt_id = trial.suggest_categorical("optimizer",range(len(opt_functional)))
    optim.Adam
    btch_sz = 2**trial.suggest_int('batch_size',4,6,1)

    training_generator = torch.utils.data.DataLoader(train_ds,
                                             batch_size=btch_sz,
                                              shuffle=True)
    cnn = CNN3()
    cnn.to(device)
    optimizer= opt_functional[opt_id](cnn.parameters(),lr=1e-5)
    print(opt_functional[opt_id])
    intermediate_value=train(trial,
                             trial.suggest_int('num_epoch',30,70,20),
                             optimizer,
                             training_generator,
                             cnn,
                             device)

    return intermediate_value

Trials which have small variance at the 5 tailing epochs pruned.

In [ ]:
study = optuna.create_study(pruner=ThresholdPruner(lower=0.001))
study.optimize(objective, n_trials=50)

[I 2020-12-11 18:22:32,270] A new study created in memory with name: no-name-37c0e16c-8328-4d68-ab2b-deed88358c0b


cuda:0
<class 'torch.optim.rmsprop.RMSprop'>
[1] loss: 2.24951 - acc 0.38775
[2] loss: 1.98884 - acc 0.60828
[3] loss: 1.79750 - acc 0.74058
[4] loss: 1.69105 - acc 0.83090
[5] loss: 1.63553 - acc 0.86800
var:  0.050075777019312664
[6] loss: 1.60462 - acc 0.88700
var:  0.019361518477749192
[7] loss: 1.58571 - acc 0.89970
var:  0.0058029859987647125
[8] loss: 1.57215 - acc 0.90935
var:  0.0017935355476223983


[I 2020-12-11 18:25:06,517] Trial 0 pruned. 


[9] loss: 1.56243 - acc 0.91647
var:  0.0006723052829720962
cuda:0
<class 'torch.optim.rmsprop.RMSprop'>
[1] loss: 2.27445 - acc 0.22833
[2] loss: 2.14819 - acc 0.35936
[3] loss: 1.99482 - acc 0.56388
[4] loss: 1.88378 - acc 0.63734
[5] loss: 1.81241 - acc 0.69868
var:  0.02868491483079652
[6] loss: 1.74389 - acc 0.77544
var:  0.020302548890050936
[7] loss: 1.69408 - acc 0.81660
var:  0.011233219640825392
[8] loss: 1.66108 - acc 0.83970
var:  0.006490704686275661
[9] loss: 1.63807 - acc 0.85509
var:  0.0038912132346668265
[10] loss: 1.62032 - acc 0.86942
var:  0.0019192207168960015


[I 2020-12-11 18:27:44,259] Trial 1 pruned. 


...

[10] loss: 1.58810 - acc 0.89664
var:  0.0008440730885649735
cuda:0
<class 'torch.optim.rmsprop.RMSprop'>
[1] loss: 2.26456 - acc 0.31920
[2] loss: 2.09881 - acc 0.49693
[3] loss: 1.93116 - acc 0.61990
[4] loss: 1.82504 - acc 0.71362
[5] loss: 1.75499 - acc 0.76594
var:  0.03439779636711606
[6] loss: 1.69749 - acc 0.81890
var:  0.020147819272752186
[7] loss: 1.65974 - acc 0.84410
var:  0.009314464715102555
[8] loss: 1.63382 - acc 0.86291
var:  0.004731058084860734
[9] loss: 1.61425 - acc 0.87622
var:  0.002496097095159025
[10] loss: 1.60070 - acc 0.88558
var:  0.0011867893695262902


[I 2020-12-11 20:23:15,122] Trial 48 pruned. 


[11] loss: 1.58882 - acc 0.89337
var:  0.0006293853197255739
cuda:0
<class 'torch.optim.rmsprop.RMSprop'>
[1] loss: 2.26705 - acc 0.33049
[2] loss: 2.10866 - acc 0.45882
[3] loss: 1.94799 - acc 0.60296
[4] loss: 1.84561 - acc 0.67859
[5] loss: 1.77292 - acc 0.74988
var:  0.032112271357151764
[6] loss: 1.72822 - acc 0.77720
var:  0.01851795409237817
[7] loss: 1.69988 - acc 0.79383
var:  0.007976567691334724
[8] loss: 1.68116 - acc 0.80495
var:  0.0034585538518976397
[9] loss: 1.66688 - acc 0.81448
var:  0.0014169164775007458


[I 2020-12-11 20:25:37,079] Trial 49 pruned. 


[10] loss: 1.64351 - acc 0.84305
var:  0.0008296100959078337


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_num_epoch,params_optimizer,state
0,0,6.723053e-04,2020-12-11 18:22:32.273226,2020-12-11 18:25:06.517541,0 days 00:02:34.244315,5,50,1,PRUNED
1,1,9.375125e-04,2020-12-11 18:25:06.519599,2020-12-11 18:27:44.259158,0 days 00:02:37.739559,6,50,1,PRUNED
2,2,6.170476e-04,2020-12-11 18:27:44.260643,2020-12-11 18:30:21.863133,0 days 00:02:37.602490,5,30,0,PRUNED
3,3,9.679335e-11,2020-12-11 18:30:21.865755,2020-12-11 18:31:30.108482,0 days 00:01:08.242727,6,30,2,PRUNED
4,4,9.900878e-04,2020-12-11 18:31:30.109665,2020-12-11 18:34:53.366075,0 days 00:03:23.256410,4,30,1,PRUNED
5,5,6.715187e-04,2020-12-11 18:34:53.367834,2020-12-11 18:37:41.464347,0 days 00:02:48.096513,6,30,1,PRUNED
6,6,7.915824e-04,2020-12-11 18:37:41.465510,2020-12-11 18:40:01.329520,0 days 00:02:19.864010,5,70,0,PRUNED
7,7,8.535845e-10,2020-12-11 18:40:01.331331,2020-12-11 18:41:20.321553,0 days 00:01:18.990222,5,70,2,PRUNED
8,8,8.082147e-10,2020-12-11 18:41:20.323194,2020-12-11 18:42:40.777492,0 days 00:01:20.454298,5,30,2,PRUNED
9,9,7.283383e-04,2020-12-11 18:42:40.778666,2020-12-11 18:45:14.735192,0 days 00:02:33.956526,6,30,1,PRUNED
